In [20]:
from sklearn.metrics import r2_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score as ROC
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import accuracy_score as ACC
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score as BACC
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.neighbors import KNeighborsClassifier as knnc
from sklearn.svm import SVC as svc
from sklearn.linear_model import LinearRegression as LR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import sys
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor as xgbr
from xgboost import XGBClassifier as xgbc
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.linear_model import LinearRegression as LinearR

import sys
sys.path.append('./')
import MLfunction





In [ ]:
massData = pd.read_csv('./TCMEIMSTOXData.csv', index_col=0).iloc[:,3:].values
print(massData.shape)
MZ_index = np.concatenate([np.arange(12001), -np.arange(12001)])[massData.sum(axis=0)!=0]
massData = massData[:,massData.sum(axis=0)!=0]
print(massData.shape)
labelTox = pd.read_csv('./TCMEIMSTOXData.csv', index_col=0).iloc[:,2].tolist()
original_data = pd.read_csv('./TCMEIMSTOXData.csv', index_col=0)


In [ ]:
original_index = original_data.index
np.random.seed(1)
new_index = np.where(np.array(labelTox)==1)[0].tolist()
new_index += np.where(np.array(labelTox)==2)[0].tolist()
new_index += np.random.choice(np.where(np.array(labelTox)==0)[0], 44, replace=False).tolist()
np.random.seed(None)
filtered_index = np.array(original_index)[new_index]
x = StandardScaler().fit_transform(massData[new_index])
# y = np.array(labelTox)[new_index].astype(np.int32)
y = 1-(np.array(labelTox)[new_index].astype(np.int32)==0)
y, np.array(labelTox)[new_index].astype(np.int32)

In [14]:
np.random.seed(1)
new_index = np.where(np.array(labelTox)==1)[0].tolist()
new_index += np.where(np.array(labelTox)==2)[0].tolist()
new_index += np.random.choice(np.where(np.array(labelTox)==0)[0], 44, replace=False).tolist()
np.random.seed(None)
filtered_index = np.array(original_index)[new_index]
x = StandardScaler().fit_transform(massData[new_index])
y = 1 - (np.array(labelTox)[new_index].astype(np.int32) == 0)

pca = PCA(n_components=3)
x_pca = pca.fit_transform(x)
mask = np.any(np.abs(x_pca) > 40, axis=1)
x_pca_filtered = x_pca[~mask]
y_filtered = y[~mask]
filtered_index_filtered = filtered_index[~mask]
pc2_values = x_pca_filtered[:, 1]
y_filtered_indices = np.where(y_filtered == 0)[0]
top2_indices = y_filtered_indices[np.argsort(np.abs(pc2_values[y_filtered_indices]))[-2:]]
x_final = np.delete(x_pca_filtered, top2_indices, axis=0)
y_final = np.delete(y_filtered, top2_indices, axis=0)
filtered_index_final = np.delete(filtered_index_filtered, top2_indices, axis=0)


final_data_df = pd.DataFrame(x_final)
final_data_df['Label'] = y_final
final_data_df['Index'] = filtered_index_final

#  CSV 
#final_data_df.to_csv('filtered_final_data.csv', index=False)
x = StandardScaler().fit_transform(massData[filtered_index_final])
y = y_final

In [ ]:
# RF
scores = []

for i in list(range(0,1000)):
    results = MLfunction.cross_val(rfc(random_state=i), x, y, random_seed=i, scoresFun=[BACC, ACC], cv=5)
    scores.append(results.scores.mean()[0])
    if results.scores.std()[1]<0.1 and results.scores.mean()[1]>0.66:
        print(results.pred_all)
        display(i, results.scores.mean(), results.scores.std(), '*'*50)


In [ ]:

parmas = {'random_state':range(300), 
          'n_estimators':range(1,200), 
          'max_depth':range(1,100), 
        #   'max_features':range(int(x.shape[1]**0.5),x.shape[1]), 
          'min_impurity_decrease':np.linspace(0,0.5,20), 
          'max_samples':list(range(1,60))+[None]
          }

parma_youhua = MLfunction.parmaOptimaze(rfc(random_state=794), [x, y], parmas, random_seed=794, cv=5, scoresFun=[BACC, ACC, F1])

In [ ]:
results = MLfunction.cross_val(rfc(max_depth=9, n_estimators=110, random_state=23), x, y, random_seed=794, scoresFun=[BACC, ACC, F1], cv=5)
print("Cross Validation Scores:")
print(results.scores)

print("\nAverage Scores:")
print(results.scores.mean())

In [ ]:


model_trained = rfc(max_depth=9, n_estimators=110, random_state=23).fit(x, y)

feature_importances = model_trained.feature_importances_
sorted_indices = np.argsort(feature_importances)[::-1]
top_xxx = 50  


top_features = MZ_index[sorted_indices][:top_xxx]
top_importances = feature_importances[sorted_indices][:top_xxx]
importance_df = pd.DataFrame({'Feature': top_features, 'Importance': top_importances})
importance_df.to_csv('feature_importance.csv', index=False)

plt.figure(figsize=[16, 9], dpi=300)
plt.bar(range(top_xxx), top_importances)
plt.xticks(range(top_xxx), top_features / 10, fontsize=20, rotation=45)
plt.yticks(fontsize=20)
plt.show()


In [ ]:
model_trained = rfc(max_depth=9, n_estimators=110, random_state=23).fit(x, y)
importances = model_trained.feature_importances_


plt.rcParams['font.family'] = 'Arial'
plt.figure(figsize=[16, 9], dpi=300)
plt.bar(range(len(importances)), importances, color='black')

top_features = np.argsort(importances)[-3:]

for idx in top_features:
    plt.bar(idx, importances[idx], color='red', width=20)

    plt.text(idx, importances[idx], f'{MZ_index[idx]/10:.1f}', ha='center', va='bottom', color='red', fontsize=12)

plt.xticks(np.arange(0, len(importances), step=1200), labels=np.round(MZ_index[::1200]/10, 1), rotation=45, fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('m/z', fontsize=40)
plt.ylabel('Importances', fontsize=40)

plt.show()


In [ ]:
def result_figure_classifier(y_kf=[], y_test=[], yLabels=None, labels=['Experiment', 'Prediction'],
                             cmap=plt.cm.Reds, name='Confusion Matrix', path='./'):

    plt.rcParams['font.family'] = "Arial"
    plt.figure(figsize=[10, 10], dpi=300)
    

    os.makedirs(f'{path}/csv/', exist_ok=True)
    os.makedirs(f'{path}/figure/', exist_ok=True)
    
    if len(y_kf) == 2:
        kfTrue = y_kf[0]
        kfPred = y_kf[1]
        acc_train = ACC(kfTrue, kfPred)
        bacc_train = BACC(kfTrue, kfPred)
        f1_train = f1_score(kfTrue, kfPred, average='weighted')

        trainPD = pd.DataFrame(columns=['True', 'Prediction', 'ACC', 'BACC', 'F1 Score'])
        trainPD['True'] = y_kf[0]
        trainPD['Prediction'] = y_kf[1]
        trainPD.loc[0, ['ACC', 'BACC', 'F1 Score']] = [acc_train, bacc_train, f1_train]
        trainPD.to_csv(f'{path}/csv/{name}_kf.csv', index=None)

    if len(y_test) == 2:
        testTrue = y_test[0]
        testPred = y_test[1]
        acc_test = ACC(testTrue, testPred)
        bacc_test = BACC(testTrue, testPred)
        f1_test = f1_score(testTrue, testPred, average='weighted')

        testPD = pd.DataFrame(columns=['True', 'Prediction', 'ACC', 'BACC', 'F1 Score'])
        testPD['True'] = y_test[0]
        testPD['Prediction'] = y_test[1]
        testPD.loc[0, ['ACC', 'BACC', 'F1 Score']] = [acc_test, bacc_test, f1_test]
        testPD.to_csv(f'{path}/csv/{name}_test.csv', index=None)

    if len(y_kf) + len(y_test) == 4:
        trueALL = np.concatenate([y_kf[0], y_test[0]])
        predALL = np.concatenate([y_kf[1], y_test[1]])

        C = confusion_matrix(trueALL, predALL, labels=range(len(yLabels))).T
        C_kf = confusion_matrix(y_kf[0], y_kf[1], labels=range(len(yLabels))).T
        C_test = confusion_matrix(y_test[0], y_test[1], labels=range(len(yLabels))).T
        plt.matshow(C, cmap=cmap, fignum=0)

        for i in range(len(C)):
            for j in range(len(C)):
                plt.annotate(f'K-fold = {C_kf[j, i]}\nTest = {C_test[j, i]}', xy=(i, j),
                             horizontalalignment='center', verticalalignment='center', fontsize=30)  
    else:
        if len(y_kf) == 2:
            trueALL = y_kf[0]
            predALL = y_kf[1]
        elif len(y_test) == 2:
            trueALL = y_test[0]
            predALL = y_test[1]
        C = confusion_matrix(trueALL, predALL, labels=range(len(yLabels))).T
        plt.matshow(C, cmap=cmap, fignum=0)

        for i in range(len(C)):
            for j in range(len(C)):
                plt.annotate(C[j, i], xy=(i, j), horizontalalignment='center', verticalalignment='center',
                             fontsize=30)  

    plt.tick_params(labelsize=30, bottom=True, top=False, labelbottom=True, labeltop=False)
    plt.xlabel(labels[0], fontsize=35, labelpad=10)
    plt.ylabel(labels[1], fontsize=35)
    plt.xticks(range(len(yLabels)), yLabels)
    plt.yticks(range(len(yLabels)), yLabels)
    plt.tight_layout()
    

    plt.savefig(f'{path}/figure/{name}.tif', dpi=800, bbox_inches='tight')
    plt.show()

    if len(y_kf) + len(y_test) == 4:
        print("ACC-kf = {:.3f}       ACC-test = {:.3f}".format(acc_train, acc_test))
        print("BACC-kf = {:.3f}      BACC-test = {:.3f}".format(bacc_train, bacc_test))
        print("F1 Score-kf = {:.3f}  F1 Score-test = {:.3f}".format(f1_train, f1_test))
    elif len(y_kf) == 2:
        print("ACC-kf = {:.3f}".format(acc_train))
        print("BACC-kf = {:.3f}".format(bacc_train))
        print("F1 Score-kf = {:.3f}".format(f1_train))
    elif len(y_test) == 2:
        print("ACC-test = {:.3f}".format(acc_test))
        print("BACC-test = {:.3f}".format(bacc_test))
        print("F1 Score-test = {:.3f}".format(f1_test))


result_figure_classifier(y_kf=[y, results.pred_all], yLabels=['Safe', 'Tox'], name='Confusion Matrix', path='./')



In [ ]:
# XGB


parmas = {'n_estimators':range(1,200),
          'learning_rate':np.arange(0.01,0.5,0.01),
          'max_depth':range(1,1000), 
        #   'max_features':range(int(x.shape[1]**0.5),x.shape[1]), 
          'gamma':np.arange(0,5,0.5), 
          'reg_alpha':np.arange(0,5,0.05),
          'reg_lambda':np.arange(0,5,0.05),
          # 'max_samples':list(range(1,60))+[None],
          }


parma_youhua = MLfunction.parmaOptimaze(xgbc(use_label_encoder=False, objective="binary:logistic"), [x, y], parmas, random_seed=794, cv=5, scoresFun=[BACC, ACC])

In [ ]:
scores = []
for i in list(range(0,1000)):
    results = MLfunction.cross_val(xgbc(random_state=i), x, y, random_seed=i, scoresFun=[BACC, ACC], cv=5)
    scores.append(results.scores.mean()[0])
    if results.scores.std()[1]<0.1 and results.scores.mean()[1]>0.66:
        print(results.pred_all)
        display(i, results.scores.mean(), results.scores.std(), '*'*50)

In [ ]:

parmas = {'n_estimators':range(1,200),
          'learning_rate':np.arange(0.01,0.5,0.01),
          'max_depth':range(1,1000), 
        #   'max_features':range(int(x.shape[1]**0.5),x.shape[1]), 
          'gamma':np.arange(0,5,0.5), 
          'reg_alpha':np.arange(0,5,0.05),
          'reg_lambda':np.arange(0,5,0.05),
          # 'max_samples':list(range(1,60))+[None],
          }

parma_youhua = MLfunction.parmaOptimaze(xgbc(use_label_encoder=False, objective="binary:logistic"), [x, y], parmas, random_seed=893, cv=5, scoresFun=[BACC, ACC])

In [ ]:


results = MLfunction.cross_val(xgbc(gamma=0.0, max_depth=5, learning_rate=0.3, n_estimators=60, reg_lambda=4.9), x, y, random_seed=893, scoresFun=[BACC, ACC, F1], cv=5)
results.scores, results.scores.mean()

In [ ]:
# knn
parmas = {'algorithm':['ball_tree', 'kd_tree', 'brute'],
          'weights':['uniform', 'distance'],
          'n_neighbors':range(1,50),
          'p':[1,2],
          'leaf_size':range(1,100)
          }

parma_youhua = MLfunction.parmaOptimaze(knnc(), [x, y], parmas, random_seed=240, cv=5, scoresFun=[BACC, ACC])

In [ ]:
from sklearn.linear_model import BayesianRidge
knnc_model = knnc(algorithm='ball_tree', leaf_size=1, n_neighbors=39)
results = MLfunction.cross_val(knnc_model, x, y, random_seed=240, scoresFun=[BACC, ACC, F1], cv=5)
results.scores, results.scores.mean()

In [ ]:
# 2 svm
parmas = {'kernel':["rbf", "poly", "sigmoid"],
          'C':np.arange(1,50,0.05),
          'gamma':np.linspace(0.0001,100/x.shape[1],200),
          }

parma_youhua = MLfunction.parmaOptimaze(svc(), [x, y], parmas, random_seed=100, cv=5, scoresFun=[BACC, ACC])

In [ ]:
svc_model = svc(C=1.7000000000000006, gamma=0.0004373368383844918, kernel='sigmoid',probability=True)
results = MLfunction.cross_val(svc_model, x, y, random_seed=100, scoresFun=[BACC, ACC, F1], cv=5)
results.scores, results.scores.mean()